<a href="https://colab.research.google.com/github/FengruiJing/HIV_Prediction/blob/main/SpatialRegressionOpenAccess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#U.S. data

##SEM

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import math

import libpysal
from libpysal.weights import util, W, Queen
from scipy.optimize import minimize
from numpy.linalg import slogdet
from scipy.stats import norm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# For parallel processing if needed
from joblib import Parallel, delayed

###############################################################################
# A. Read shapefile and required fields, ensuring unique FIPS codes
###############################################################################
shp_path = "US_HIV_Merged_total.shp"  # Change to your path
gdf = gpd.read_file(shp_path)

if 'FIPS' not in gdf.columns:
    raise KeyError("Shapefile is missing the 'FIPS' column")
gdf['FIPS'] = gdf['FIPS'].astype(str).str.lstrip('0')
gdf = gdf.drop_duplicates(subset=['FIPS'])

# Extract required fields (adjust as necessary)
hiv_df = gdf[['FIPS', 'GonorrheaR', 'ChlamydiaR', 'PercenHIVp',
              'Population', 'UrbanRural', 'Female', 'Old', 'Black', 'Noinsuranc',
              'Poverty', 'crime16to1', 'Dissimilar', 'X', 'Y', 'geometry']].copy()
print("Loaded data (first 5 rows):")
print(hiv_df.head())

###############################################################################
# B. Construct four spatial weight matrices helper functions
###############################################################################
def subset_W(w, subset_ids):
    """
    From the given W object, retain only rows and columns whose IDs are in subset_ids.
    Returns a new W object.
    """
    from libpysal.weights import W
    subset_ids = set(subset_ids)
    new_neighbors = {}
    new_weights = {}
    for i in w.id_order:
        if i not in subset_ids:
            continue
        old_neighbors = w.neighbors[i]
        old_weights = w.weights[i]
        filtered_neighbors = []
        filtered_weights = []
        for nb, wt in zip(old_neighbors, old_weights):
            if nb in subset_ids:
                filtered_neighbors.append(nb)
                filtered_weights.append(wt)
        new_neighbors[i] = filtered_neighbors
        new_weights[i] = filtered_weights
    return W(new_neighbors, new_weights, ids=list(subset_ids))

def build_distance_matrix(coords, max_dist_km=200):
    """
    Build an inverse-distance matrix with cutoff. The matrix is row-normalized.
    max_dist_km: Distances beyond this are set to 0.
    """
    distance = np.linalg.norm(coords[:, None, :] - coords[None, :, :], axis=2)
    epsilon = 1e-10

    # Apply cutoff: distance <= max_dist_km and > 0
    mask = (distance <= max_dist_km) & (distance > 0)
    inv_dist = np.where(mask, 1.0 / (distance + epsilon), 0.0)
    # Set diagonal to zero
    np.fill_diagonal(inv_dist, 0)

    # Row normalization
    row_sums = inv_dist.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1
    norm_dist = inv_dist / row_sums
    return norm_dist

###############################################################################
# Read/Construct PCI-based (w), SCI-based (w1), Distance-based (w2, with cutoff), Queen-based (w3)
###############################################################################

### 1) PCI-based w
pci_file_path = "US_County_PCI_2019.csv"  # Change to your path
pci_df = pd.read_csv(pci_file_path)
pci_df['place_i'] = pci_df['place_i'].astype(str).str.lstrip('0')
pci_df['place_j'] = pci_df['place_j'].astype(str).str.lstrip('0')
filtered_pci_df = pci_df[
    pci_df['place_i'].isin(hiv_df['FIPS']) &
    pci_df['place_j'].isin(hiv_df['FIPS'])
].copy()
adj_matrix_pci = filtered_pci_df.pivot_table(
    index='place_i', columns='place_j', values='pci', fill_value=0
)
np.fill_diagonal(adj_matrix_pci.values, 0)
common_ids_pci = set(hiv_df['FIPS']).intersection(adj_matrix_pci.index)
adj_matrix_pci = adj_matrix_pci.reindex(index=common_ids_pci, columns=common_ids_pci, fill_value=0)
mat_val_pci = adj_matrix_pci.values.astype(float)
row_sum_pci = mat_val_pci.sum(axis=1, keepdims=True)
row_sum_pci[row_sum_pci == 0] = 1
mat_norm_pci = mat_val_pci / row_sum_pci
fips_list_pci = list(adj_matrix_pci.index)
w = libpysal.weights.util.full2W(mat_norm_pci, ids=fips_list_pci)
print("PCI-based w.n =", w.n)

### 2) SCI-based w1
sci_file_path = "processed_sci_summary.csv"
sci_df = pd.read_csv(sci_file_path)
sci_df['user_loc'] = sci_df['user_loc'].astype(str).str.lstrip('0')
sci_df['tfr_loc'] = sci_df['tfr_loc'].astype(str).str.lstrip('0')
adj_matrix_sci = sci_df.pivot_table(
    index='user_loc', columns='tfr_loc', values='tscaled_sci', fill_value=0
)
# Symmetrize the matrix
sci_np = adj_matrix_sci.values
sci_np = sci_np + sci_np.T - np.diag(sci_np.diagonal())
adj_matrix_sci = pd.DataFrame(sci_np, index=adj_matrix_sci.index, columns=adj_matrix_sci.columns)

np.fill_diagonal(adj_matrix_sci.values, 0)
common_ids_sci = set(hiv_df['FIPS']).intersection(adj_matrix_sci.index)
adj_matrix_sci = adj_matrix_sci.reindex(index=common_ids_sci, columns=common_ids_sci, fill_value=0)
mat_val_sci = adj_matrix_sci.values.astype(float)
row_sum_sci = mat_val_sci.sum(axis=1, keepdims=True)
row_sum_sci[row_sum_sci == 0] = 1
mat_norm_sci = mat_val_sci / row_sum_sci
fips_list_sci = list(adj_matrix_sci.index)
w1 = libpysal.weights.util.full2W(mat_norm_sci, ids=fips_list_sci)
print("SCI-based w1.n =", w1.n)

### 3) Inverse distance based w2 (with cutoff)
coords = hiv_df[['X', 'Y']].values
norm_dist_mat = build_distance_matrix(coords, max_dist_km=200)  # Adjust parameter if needed
fips_list_dist = hiv_df['FIPS'].tolist()
w2 = libpysal.weights.util.full2W(norm_dist_mat, ids=fips_list_dist)
print("Distance-based w2.n =", w2.n)

### 4) Queen-based w3
hiv_df_proj = hiv_df.to_crs(epsg=3857)
w3 = Queen.from_dataframe(hiv_df_proj, ids=hiv_df_proj['FIPS'])
w3.transform = 'R'
print("Queen-based w3.n =", w3.n)

### Harmonize all IDs
common_all = set(w.id_order) & set(w1.id_order) & set(w2.id_order) & set(w3.id_order) & set(hiv_df['FIPS'])
print("Common IDs size:", len(common_all))
hiv_df = hiv_df[hiv_df['FIPS'].isin(common_all)].copy()
print("Final data size after intersection:", len(hiv_df))
w  = subset_W(w, common_all)
w1 = subset_W(w1, common_all)
w2 = subset_W(w2, common_all)
w3 = subset_W(w3, common_all)
print("Final w.n =", w.n)
print("Final w1.n =", w1.n)
print("Final w2.n =", w2.n)
print("Final w3.n =", w3.n)
N = len(hiv_df)
print("N =", N)

###############################################################################
# C. Construct y and X, and drop rows with NaNs
###############################################################################
df_temp = hiv_df[['FIPS', 'GonorrheaR', 'ChlamydiaR', 'PercenHIVp',
                  'Population', 'UrbanRural', 'Female', 'Old', 'Black',
                  'Noinsuranc', 'Poverty', 'crime16to1', 'Dissimilar',
                  'X', 'Y', 'geometry']].copy()

print("NaN sum before dropna:\n", df_temp.isna().sum())
df_temp = df_temp.dropna(axis=0)
print("After dropna, row count:", df_temp.shape[0])

valid_ids = set(df_temp['FIPS'])
y = df_temp['PercenHIVp'].values  # Dependent variable (example: PercenHIVp)
X_cols = ['Population', 'UrbanRural', 'Female', 'Old', 'Black',
          'Noinsuranc', 'Poverty', 'crime16to1', 'Dissimilar']
X_raw = df_temp[X_cols].values
N = X_raw.shape[0]
X = np.hstack([np.ones((N, 1)), X_raw])
print("After dropna, final ID count:", len(valid_ids))
print("y.shape =", y.shape, "X.shape =", X.shape)

# Subset the 4 weight objects using valid IDs
w  = subset_W(w, valid_ids)
w1 = subset_W(w1, valid_ids)
w2 = subset_W(w2, valid_ids)
w3 = subset_W(w3, valid_ids)
print("After dropna, filtered weights sizes:")
print("w.n =", w.n, "w1.n =", w1.n, "w2.n =", w2.n, "w3.n =", w3.n)

###############################################################################
# D. Define OLS and multi-weight SEM (with approx Hessian)
###############################################################################
def fit_ols(y, X):
    N, K = X.shape
    beta_ols, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    y_hat = X @ beta_ols
    residuals = y - y_hat
    sse = residuals @ residuals
    sigma2 = sse / (N - K)
    logL = -(N / 2) * np.log(2 * np.pi * sigma2) - (sse / (2 * sigma2))
    AIC = -2 * logL + 2 * K
    AICc = AIC + (2 * K * (K + 1)) / (N - K - 1) if (N - K - 1) > 0 else np.nan
    return {
        'model': 'OLS',
        'beta': beta_ols,
        'logL': logL,
        'AIC': AIC,
        'AICc': AICc,
        'residuals': residuals
    }

def neg_loglike_sem(params, y, X, W_list):
    N, K = X.shape
    m = len(W_list)
    lam = params[:m]
    beta = params[m:]

    I = np.eye(N)
    A = I.copy()
    for i in range(m):
        A -= lam[i] * W_list[i]

    sign, logdetA = np.linalg.slogdet(A)
    if sign <= 0:
        return 1e12

    e = y - X @ beta
    v = A @ e
    dof = N - (K + m)
    if dof <= 0:
        return 1e12
    sse = v @ v
    sigma2 = sse / dof
    if sigma2 <= 0:
        return 1e12

    logL = logdetA - (N / 2) * np.log(2 * np.pi * sigma2) - (sse / (2 * sigma2))
    return -logL

def fit_sem_multi(y, X, W_list):
    N, K = X.shape
    m = len(W_list)
    # Initial guess using OLS estimates
    beta_ols, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    init_params = np.concatenate([np.zeros(m), beta_ols])
    bounds = [(-0.99, 0.99)] * m + [(-np.inf, np.inf)] * K

    res = minimize(
        neg_loglike_sem,
        init_params,
        args=(y, X, W_list),
        method='L-BFGS-B',
        bounds=bounds
    )
    param_hat = res.x
    lam_hat = param_hat[:m]
    beta_hat = param_hat[m:]
    neg_ll = res.fun
    logL = -neg_ll
    p_total = m + K

    AIC = -2 * logL + 2 * p_total
    AICc = AIC + (2 * p_total * (p_total + 1)) / (N - p_total - 1) if (N - p_total - 1) > 0 else np.nan

    # Residuals calculation
    I = np.eye(N)
    A = I.copy()
    for i in range(m):
        A -= lam_hat[i] * W_list[i]
    e = y - X @ beta_hat
    v = A @ e

    # Hessian approximation for standard errors
    if hasattr(res, 'hess_inv'):
        hess_inv_approx = res.hess_inv
        if hasattr(hess_inv_approx, 'todense'):
            hess_inv_approx = hess_inv_approx.todense()
        if hess_inv_approx.shape == (p_total, p_total):
            se = np.sqrt(np.diag(hess_inv_approx))
            t_stats = param_hat / se
            p_vals = 2 * (1 - norm.cdf(np.abs(t_stats)))
        else:
            se = np.full(p_total, np.nan)
            t_stats = np.full(p_total, np.nan)
            p_vals = np.full(p_total, np.nan)
    else:
        se = np.full(p_total, np.nan)
        t_stats = np.full(p_total, np.nan)
        p_vals = np.full(p_total, np.nan)

    return {
        'model': f'SEM_{m}_weights',
        'lam': lam_hat,
        'beta': beta_hat,
        'logL': logL,
        'AIC': AIC,
        'AICc': AICc,
        'residuals': v,
        'converged': res.success,
        'message': res.message,
        'se_hessian': se,
        't_hessian': t_stats,
        'p_hessian': p_vals
    }

###############################################################################
# E. Flatten matrices & check multicollinearity
###############################################################################
def flatten_W(W_mat):
    """
    Flatten an NxN matrix (excluding the diagonal) into a vector.
    """
    N = W_mat.shape[0]
    mask = ~np.eye(N, dtype=bool)
    return W_mat[mask]

def check_correlation_among(W_list, names=None):
    """
    Flatten all matrices in W_list and combine into one DataFrame,
    then output the correlation matrix and VIF.
    Skip VIF if only one column is present.
    """
    if names is None:
        names = [f"W{i}" for i in range(len(W_list))]

    data_dict = {}
    for i, wmat in enumerate(W_list):
        vec = flatten_W(wmat)
        data_dict[names[i]] = vec
    df = pd.DataFrame(data_dict)

    print("\n------ Checking correlation among these matrices ------")
    # If only one column, cannot calculate VIF
    if df.shape[1] < 2:
        single_col = df.columns[0]
        print(f"(Only 1 column: '{single_col}'. No correlation or VIF computed for single column scenario.)")
        return
    else:
        corr = df.corr()
        print("Correlation matrix:")
        print(corr)

        print("\nVIF:")
        X_vif = df.values
        for i, col in enumerate(df.columns):
            vif_val = variance_inflation_factor(X_vif, i)
            print(f"{col}: {vif_val:.3f}")
    print("-------------------------------------------------------")

###############################################################################
# F. Stepwise model construction: single and pairwise combinations
###############################################################################
arr_w, _ = w.full()
arr_w1, _ = w1.full()
arr_w2, _ = w2.full()
arr_w3, _ = w3.full()

all_matrices = {
    "WP": arr_w,   # PCI-based
    "WS": arr_w1,  # SCI-based
    "WD": arr_w2,  # Distance-based
    "WQ": arr_w3   # Queen-based
}

# 1) Single matrix models
from itertools import combinations

single_tasks = []
for k in all_matrices:
    single_tasks.append((f"SEM_1_{k}", [all_matrices[k]]))

# 2) Two-matrix combinations
pair_tasks = []
keys_list = list(all_matrices.keys())
for combo in combinations(keys_list, 2):
    name_combo = "_".join(combo)
    arr_list = [all_matrices[c] for c in combo]
    pair_tasks.append((f"SEM_2_{name_combo}", arr_list))

task_list = single_tasks + pair_tasks

###############################################################################
# G. Execute SEM fitting and output results
###############################################################################
models = {}
print("\n================ Starting stepwise model testing =================\n")

for (model_name, W_list) in task_list:
    # First: check multicollinearity among the weight matrices
    matrix_names = [f"{model_name}_mat{i}" for i in range(len(W_list))]
    check_correlation_among(W_list, names=matrix_names)

    # Fit the SEM using the provided weight matrices
    resdict = fit_sem_multi(y, X, W_list)
    models[model_name] = resdict

# Finally, add OLS as a benchmark model
ols_res = fit_ols(y, X)
models["OLS"] = ols_res

print("\n=== Model: OLS ===")
print(f"logL = {ols_res['logL']:.3f}, AIC = {ols_res['AIC']:.3f}, AICc = {ols_res['AICc']:.3f}")

for name, res in models.items():
    if name == "OLS":
        continue
    print(f"\n=== Model: {name} ===")
    print(f"logL = {res['logL']:.3f}, AIC = {res['AIC']:.3f}, AICc = {res['AICc']:.3f}")
    print("Converged?", res['converged'], "| Message:", res['message'])
    lam_ = res['lam']
    beta_ = res['beta']
    se_all = res['se_hessian']
    t_all = res['t_hessian']
    p_all = res['p_hessian']
    m_ = len(lam_)

    print("Spatial parameters (lambda):")
    for i in range(m_):
        print(f"  lam[{i+1}] = {lam_[i]:.6f}, SE = {se_all[i]:.6f}, t = {t_all[i]:.3f}, p = {p_all[i]:.4f}")

    print("Regression coefficients (beta):")
    for j in range(len(beta_)):
        idx = m_ + j
        print(f"  beta[{j}] = {beta_[j]:.6f}, SE = {se_all[idx]:.6f}, t = {t_all[idx]:.3f}, p = {p_all[idx]:.4f}")

print("\n================ Completed: Stepwise model testing, multicollinearity check & distance cutoff =================\n")


##SLM

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

import libpysal
from libpysal.weights import util, W, Queen
from scipy.optimize import minimize
from numpy.linalg import slogdet
from scipy.stats import norm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Use joblib for parallel processing if needed
from joblib import Parallel, delayed
from itertools import combinations

###############################################################################
# A. Read Shapefile and Required Fields, Ensuring Unique FIPS Codes
###############################################################################
shp_path = "US_HIV_Merged_total.shp"  # Change to your own path
gdf = gpd.read_file(shp_path)

if 'FIPS' not in gdf.columns:
    raise KeyError("Shapefile is missing the 'FIPS' column")
gdf['FIPS'] = gdf['FIPS'].astype(str).str.lstrip('0')
gdf = gdf.drop_duplicates(subset=['FIPS'])

# Extract required fields (adjust as needed)
hiv_df = gdf[['FIPS', 'GonorrheaR', 'ChlamydiaR', 'PercenHIVp',
              'Population', 'UrbanRural', 'Female', 'Old', 'Black', 'Noinsuranc',
              'Poverty', 'crime16to1', 'Dissimilar', 'X', 'Y', 'geometry']].copy()
print("Loaded data (first 5 rows):")
print(hiv_df.head())

###############################################################################
# B. Helper Functions: subset_W, build_distance_matrix
###############################################################################
def subset_W(w, subset_ids):
    """
    From the given W object, keep only the rows and columns corresponding
    to the IDs in subset_ids, and return a new W object.
    """
    from libpysal.weights import W
    subset_ids = set(subset_ids)
    new_neighbors = {}
    new_weights = {}
    for i in w.id_order:
        if i not in subset_ids:
            continue
        old_neighbors = w.neighbors[i]
        old_weights = w.weights[i]
        filtered_neighbors = []
        filtered_weights = []
        for nb, wt in zip(old_neighbors, old_weights):
            if nb in subset_ids:
                filtered_neighbors.append(nb)
                filtered_weights.append(wt)
        new_neighbors[i] = filtered_neighbors
        new_weights[i] = filtered_weights
    return W(new_neighbors, new_weights, ids=list(subset_ids))

def build_distance_matrix(coords, max_dist_km=200):
    """
    Generate an inverse-distance matrix with a cutoff and perform row normalization.
    Distances greater than max_dist_km are set to 0, and the diagonal is set to 0.
    Returns an (N x N) numpy array.
    """
    distance = np.linalg.norm(coords[:, None, :] - coords[None, :, :], axis=2)
    epsilon = 1e-10

    # Apply cutoff: only distances <= max_dist_km and > 0
    mask = (distance <= max_dist_km) & (distance > 0)
    inv_dist = np.where(mask, 1.0/(distance + epsilon), 0.0)
    np.fill_diagonal(inv_dist, 0)

    # Row normalization
    row_sums = inv_dist.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1
    norm_dist = inv_dist / row_sums
    return norm_dist

###############################################################################
# C. Construct the 4 Spatial Weights (PCI, SCI, Distance, Queen)
###############################################################################
# 1) PCI-based Weight (w)
pci_file_path = "US_County_PCI_2019.csv"  # Change your path
pci_df = pd.read_csv(pci_file_path)
pci_df['place_i'] = pci_df['place_i'].astype(str).str.lstrip('0')
pci_df['place_j'] = pci_df['place_j'].astype(str).str.lstrip('0')
filtered_pci_df = pci_df[
    pci_df['place_i'].isin(hiv_df['FIPS']) & pci_df['place_j'].isin(hiv_df['FIPS'])
].copy()
adj_matrix_pci = filtered_pci_df.pivot_table(
    index='place_i', columns='place_j', values='pci', fill_value=0
)
np.fill_diagonal(adj_matrix_pci.values, 0)
common_ids_pci = set(hiv_df['FIPS']).intersection(adj_matrix_pci.index)
adj_matrix_pci = adj_matrix_pci.reindex(index=common_ids_pci, columns=common_ids_pci, fill_value=0)
mat_val_pci = adj_matrix_pci.values.astype(float)
row_sum_pci = mat_val_pci.sum(axis=1, keepdims=True)
row_sum_pci[row_sum_pci==0] = 1
mat_norm_pci = mat_val_pci / row_sum_pci
fips_list_pci = list(adj_matrix_pci.index)
w = libpysal.weights.util.full2W(mat_norm_pci, ids=fips_list_pci)
print("PCI-based w.n =", w.n)

# 2) SCI-based Weight (w1)
sci_file_path = "processed_sci_summary.csv"  # Change your path
sci_df = pd.read_csv(sci_file_path)
sci_df['user_loc'] = sci_df['user_loc'].astype(str).str.lstrip('0')
sci_df['tfr_loc'] = sci_df['tfr_loc'].astype(str).str.lstrip('0')
adj_matrix_sci = sci_df.pivot_table(
    index='user_loc', columns='tfr_loc', values='tscaled_sci', fill_value=0
)
# Symmetrize the matrix
sci_np = adj_matrix_sci.values
sci_np = sci_np + sci_np.T - np.diag(sci_np.diagonal())
adj_matrix_sci = pd.DataFrame(sci_np, index=adj_matrix_sci.index, columns=adj_matrix_sci.columns)
np.fill_diagonal(adj_matrix_sci.values, 0)
common_ids_sci = set(hiv_df['FIPS']).intersection(adj_matrix_sci.index)
adj_matrix_sci = adj_matrix_sci.reindex(index=common_ids_sci, columns=common_ids_sci, fill_value=0)
mat_val_sci = adj_matrix_sci.values.astype(float)
row_sum_sci = mat_val_sci.sum(axis=1, keepdims=True)
row_sum_sci[row_sum_sci==0] = 1
mat_norm_sci = mat_val_sci / row_sum_sci
fips_list_sci = list(adj_matrix_sci.index)
w1 = libpysal.weights.util.full2W(mat_norm_sci, ids=fips_list_sci)
print("SCI-based w1.n =", w1.n)

# 3) Distance-based Weight (w2) with a 200km cutoff
coords = hiv_df[['X', 'Y']].values
norm_dist_mat = build_distance_matrix(coords, max_dist_km=200)
fips_list_dist = hiv_df['FIPS'].tolist()
w2 = libpysal.weights.util.full2W(norm_dist_mat, ids=fips_list_dist)
print("Distance-based w2.n =", w2.n)

# 4) Queen-based Weight (w3)
hiv_df_proj = hiv_df.to_crs(epsg=3857)
w3 = Queen.from_dataframe(hiv_df_proj, ids=hiv_df_proj['FIPS'])
w3.transform = 'R'
print("Queen-based w3.n =", w3.n)

# Harmonize IDs across all weight matrices
common_all = set(w.id_order) & set(w1.id_order) & set(w2.id_order) & set(w3.id_order) & set(hiv_df['FIPS'])
print("Common IDs size:", len(common_all))
hiv_df = hiv_df[hiv_df['FIPS'].isin(common_all)].copy()
print("Final data size after intersection:", len(hiv_df))
w = subset_W(w, common_all)
w1 = subset_W(w1, common_all)
w2 = subset_W(w2, common_all)
w3 = subset_W(w3, common_all)
print("Final w.n =", w.n, "w1.n =", w1.n, "w2.n =", w2.n, "w3.n =", w3.n)
N = len(hiv_df)
print("N =", N)

###############################################################################
# D. Construct y and X, and drop rows with NaN values
###############################################################################
df_temp = hiv_df[['FIPS', 'GonorrheaR', 'ChlamydiaR', 'PercenHIVp',
                  'Population', 'UrbanRural', 'Female', 'Old', 'Black',
                  'Noinsuranc', 'Poverty', 'crime16to1', 'Dissimilar',
                  'X', 'Y', 'geometry']].copy()

print("NaN sum before dropna:\n", df_temp.isna().sum())
df_temp = df_temp.dropna(axis=0)
print("After dropna, row count:", df_temp.shape[0])

valid_ids = set(df_temp['FIPS'])
y = df_temp['PercenHIVp'].values  # Dependent variable
X_cols = ['Population', 'UrbanRural', 'Female', 'Old', 'Black',
          'Noinsuranc', 'Poverty', 'crime16to1', 'Dissimilar']
X_raw = df_temp[X_cols].values
N = X_raw.shape[0]
X = np.hstack([np.ones((N, 1)), X_raw])
print("After dropna, final ID count:", len(valid_ids))
print("y.shape =", y.shape, "X.shape =", X.shape)

# Subset the 4 spatial weights using valid IDs
w = subset_W(w, valid_ids)
w1 = subset_W(w1, valid_ids)
w2 = subset_W(w2, valid_ids)
w3 = subset_W(w3, valid_ids)
print("After dropna, filtered weights sizes:")
print("w.n =", w.n, "w1.n =", w1.n, "w2.n =", w2.n, "w3.n =", w3.n)

###############################################################################
# E. Define OLS and Multi-Weight SLM (Spatial Lag Model) MLE
###############################################################################
def fit_ols(y, X):
    N, K = X.shape
    beta_ols, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    y_hat = X @ beta_ols
    residuals = y - y_hat
    sse = residuals @ residuals
    sigma2 = sse / (N - K)
    logL = -(N / 2) * np.log(2 * np.pi * sigma2) - (sse / (2 * sigma2))
    AIC = -2 * logL + 2 * K
    AICc = AIC + (2 * K * (K + 1)) / (N - K - 1) if (N - K - 1) > 0 else np.nan
    return {
        'model': 'OLS',
        'beta': beta_ols,
        'logL': logL,
        'AIC': AIC,
        'AICc': AICc,
        'residuals': residuals
    }

def neg_loglike_slm(params, y, X, W_list):
    """
    Negative log-likelihood for the Spatial Lag Model (SLM):
       y = Σ_i (rho_i * W_i) y + X β + e
       => A = I - Σ_i (rho_i * W_i)
       => e = A y - X β
       => SSE = e'e, sigma2 = SSE / dof
       => log-likelihood = log|A| - (N/2)*ln(2πsigma2) - SSE/(2sigma2)
    """
    N, K = X.shape
    m = len(W_list)
    rho = params[:m]
    beta = params[m:]

    I = np.eye(N)
    A = I.copy()
    for i in range(m):
        A -= rho[i] * W_list[i]

    sign, logdetA = np.linalg.slogdet(A)
    if sign <= 0:
        return 1e12

    # Compute residuals: e = A y - X β
    e = A @ y - X @ beta
    dof = N - (K + m)
    if dof <= 0:
        return 1e12
    sse = e @ e
    sigma2 = sse / dof
    if sigma2 <= 0:
        return 1e12

    logL = logdetA - (N / 2) * np.log(2 * np.pi * sigma2) - (sse / (2 * sigma2))
    return -logL

def fit_slm_multi(y, X, W_list):
    """
    Multi-weight SLM MLE: y = Σ_i (rho_i * W_i) y + X β + e
    """
    N, K = X.shape
    m = len(W_list)
    # Initial guess: rho=0, beta = OLS estimates
    beta_ols, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    init_params = np.concatenate([np.zeros(m), beta_ols])
    bounds = [(-0.99, 0.99)] * m + [(-np.inf, np.inf)] * K

    res = minimize(
        neg_loglike_slm,
        init_params,
        args=(y, X, W_list),
        method='L-BFGS-B',
        bounds=bounds
    )
    param_hat = res.x
    rho_hat = param_hat[:m]
    beta_hat = param_hat[m:]
    neg_ll = res.fun
    logL = -neg_ll
    p_total = m + K

    AIC = -2 * logL + 2 * p_total
    AICc = AIC + (2 * p_total * (p_total + 1)) / (N - p_total - 1) if (N - p_total - 1) > 0 else np.nan

    # Residuals: e = A y - X β
    I = np.eye(N)
    A = I.copy()
    for i in range(m):
        A -= rho_hat[i] * W_list[i]
    e = A @ y - X @ beta_hat

    # Approximate Hessian for standard errors
    if hasattr(res, 'hess_inv'):
        hess_inv_approx = res.hess_inv
        if hasattr(hess_inv_approx, 'todense'):
            hess_inv_approx = hess_inv_approx.todense()
        if hess_inv_approx.shape == (p_total, p_total):
            se = np.sqrt(np.diag(hess_inv_approx))
            t_stats = param_hat / se
            p_vals = 2 * (1 - norm.cdf(np.abs(t_stats)))
        else:
            se = np.full(p_total, np.nan)
            t_stats = np.full(p_total, np.nan)
            p_vals = np.full(p_total, np.nan)
    else:
        se = np.full(p_total, np.nan)
        t_stats = np.full(p_total, np.nan)
        p_vals = np.full(p_total, np.nan)

    return {
        'model': f'SLM_{m}_weights',
        'rho': rho_hat,
        'beta': beta_hat,
        'logL': logL,
        'AIC': AIC,
        'AICc': AICc,
        'residuals': e,
        'converged': res.success,
        'message': res.message,
        'se_hessian': se,
        't_hessian': t_stats,
        'p_hessian': p_vals
    }

###############################################################################
# F. Multicollinearity Check Functions
###############################################################################
def flatten_W(W_mat):
    """
    Flatten an NxN matrix (excluding the diagonal) into a vector.
    """
    N = W_mat.shape[0]
    mask = ~np.eye(N, dtype=bool)
    return W_mat[mask]

def check_correlation_among(W_list, names=None):
    if names is None:
        names = [f"W{i}" for i in range(len(W_list))]
    data_dict = {}
    for i, wmat in enumerate(W_list):
        vec = flatten_W(wmat)
        data_dict[names[i]] = vec
    df = pd.DataFrame(data_dict)

    print("\n------ Checking correlation among these matrices ------")
    if df.shape[1] < 2:
        single_col = df.columns[0]
        print(f"(Only 1 column: '{single_col}'. No correlation or VIF for single column scenario.)")
        return
    else:
        corr = df.corr()
        print("Correlation matrix:")
        print(corr)

        print("\nVIF:")
        X_vif = df.values
        for i, col in enumerate(df.columns):
            vif_val = variance_inflation_factor(X_vif, i)
            print(f"{col}: {vif_val:.3f}")
    print("-------------------------------------------------------")

###############################################################################
# G. Generate Single and Pairwise Combinations, Fit SLM, and Output Results
###############################################################################
arr_w, _ = w.full()
arr_w1, _ = w1.full()
arr_w2, _ = w2.full()
arr_w3, _ = w3.full()

all_matrices = {
    "WP": arr_w,  # PCI
    "WS": arr_w1, # SCI
    "WD": arr_w2, # Distance
    "WQ": arr_w3  # Queen
}

# 1) Single-matrix models
single_tasks = []
for k in all_matrices:
    single_tasks.append((f"SLM_1_{k}", [all_matrices[k]]))

# 2) Two-matrix combinations
pair_tasks = []
keys_list = list(all_matrices.keys())
for combo in combinations(keys_list, 2):
    name_combo = "_".join(combo)
    arr_list = [all_matrices[c] for c in combo]
    pair_tasks.append((f"SLM_2_{name_combo}", arr_list))

task_list = single_tasks + pair_tasks

models = {}
print("\n================ Starting Stepwise SLM Model Testing =================\n")

# Execute SLM fitting for each task
for (model_name, W_list) in task_list:
    matrix_names = [f"{model_name}_mat{i}" for i in range(len(W_list))]
    check_correlation_among(W_list, names=matrix_names)

    # Fit the multi-weight SLM
    resdict = fit_slm_multi(y, X, W_list)
    models[model_name] = resdict

# OLS as benchmark
ols_res = fit_ols(y, X)
models["OLS"] = ols_res

print("\n=== Model: OLS ===")
print(f"logL = {ols_res['logL']:.3f}, AIC = {ols_res['AIC']:.3f}, AICc = {ols_res['AICc']:.3f}")

# Print results for each model
for name, res in models.items():
    if name == "OLS":
        continue
    print(f"\n=== Model: {name} ===")
    print(f"logL = {res['logL']:.3f}, AIC = {res['AIC']:.3f}, AICc = {res['AICc']:.3f}")
    print("Converged?", res['converged'], "| Message:", res['message'])

    rho_ = res['rho']
    beta_ = res['beta']
    se_ = res['se_hessian']
    t_ = res['t_hessian']
    p_ = res['p_hessian']
    m_ = len(rho_)

    print("Spatial parameters (rho):")
    for i in range(m_):
        print(f"  rho[{i+1}] = {rho_[i]:.6f}, SE = {se_[i]:.6f}, t = {t_[i]:.3f}, p = {p_[i]:.4f}")

    print("Regression coefficients (beta):")
    for j in range(len(beta_)):
        idx = m_ + j
        print(f"  beta[{j}] = {beta_[j]:.6f}, SE = {se_[idx]:.6f}, t = {t_[idx]:.3f}, p = {p_[idx]:.4f}")

print("\n================ Completed: Multi-weight SLM Model Fitting & Checks =================\n")


##Disease types

For different diseases, simply modify the dependent variable assignment in the code above.

If you are modeling HIV prevalence, use:

"y = df_temp['PercenHIVp'].values  # Dependent variable"

If you are modeling Gonorrhea rates, use:

"y = df_temp['GonorrheaR'].values  # Dependent variable"

If you are modeling Chlamydia rates, use:

"y = df_temp['ChlamydiaR'].values  # Dependent variable"

#CBSA data

In [ ]:
For CBSA data, simply modify the dataset assignment in the code above:

shp_path = "CBSA_HIV_Merged.shp"

#DeepSouth Data

In [ ]:
For Deep South data, simply modify the dataset assignment in the code above:

shp_path = "DeepSouth_HIV_Merged.shp"